In [1]:
import cv2
cv2.__version__

'3.3.1'

In [2]:
import sys
sys.version

'3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]'

In [3]:
import numpy as np
print(np.__version__)



import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk
import time
import pandas as pd

import skimage.io as io

import os

import imageio

1.14.1


In [4]:
#vidPath = "D:/BeeTwoFlowerExperiment/2018_02_27__14_04_52_556/2018_02_27__14_04_55_680_cam1-0000.avi"
vidPath = r"D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52_556\2018_02_27__14_04_55_680_cam1-0000.avi"
os.path.isfile(vidPath)

True

In [5]:
cap = cv2.VideoCapture(vidPath)


length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

print(length, width, height, fps)
cap.release()

546 1280 1024 10.0


In [6]:
def load_imgs(vidPath, start, stop):
    cap = cv2.VideoCapture(vidPath)
    imgs = []
    for ff in range(length):
        
        
        ret, frame = cap.read()
        
        if np.mod(ff, 10) == 0:
            print(ff)
        

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        imgs.append(img)
        
        if ff > stop:
            break
            
    cap.release()
    return imgs

In [7]:
imgs = load_imgs(vidPath, 0, 50)

0
10
20
30
40
50


In [8]:
io.imshow(imgs[0])

In [9]:
r = cv2.selectROI(imgs[0], showCrosshair = True, fromCenter = False)
cv2.destroyAllWindows()

In [10]:
r

(484, 571, 173, 179)

In [11]:
# possible opencv events
events = [i for i in dir(cv2) if 'EVENT' in i]
print( events )

['EVENT_FLAG_ALTKEY', 'EVENT_FLAG_CTRLKEY', 'EVENT_FLAG_LBUTTON', 'EVENT_FLAG_MBUTTON', 'EVENT_FLAG_RBUTTON', 'EVENT_FLAG_SHIFTKEY', 'EVENT_LBUTTONDBLCLK', 'EVENT_LBUTTONDOWN', 'EVENT_LBUTTONUP', 'EVENT_MBUTTONDBLCLK', 'EVENT_MBUTTONDOWN', 'EVENT_MBUTTONUP', 'EVENT_MOUSEHWHEEL', 'EVENT_MOUSEMOVE', 'EVENT_MOUSEWHEEL', 'EVENT_RBUTTONDBLCLK', 'EVENT_RBUTTONDOWN', 'EVENT_RBUTTONUP']


In [ ]:
img.shape

In [97]:
# mouse hover and draw

ix,iy = -1,-1
drawing  = False
moving = False
buttonRelease = False
# mouse callback function
def draw_circle(event,x,y,flags,param):
    global ix,iy, drawing, moving, im3, buttonRelease, xx, yy, im2
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y
    elif event == cv2.EVENT_MOUSEMOVE:
        moving = True
        im3 = img
        #cv2.circle(img,(x,y),10,(255,0,0, 0.1),-1)
        cv2.line(img,(img.shape[1],y),(0,y),  color = (0,0,0))
        cv2.line(img,(x,0),(x,img.shape[0]),  color = (0,0,0))
        if drawing == True:
            cv2.rectangle(img,(ix,iy),(x,y),(100,100,255),1)

    elif event == cv2.EVENT_LBUTTONUP:
        buttonRelease = True
        #moving = True
        drawing = False
        cv2.rectangle(im2,(ix,iy),(x,y),(0,255,0),1)
        xx, yy = x,y
        
    
img = imgs[0].copy()
im2 = img.copy()
im3 = img.copy()
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)
rect = []
ctr = 0
behavior = ""

while(1):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img,'frame:' + str(ctr),(1,30), font, 1,(255,255,255),2,cv2.LINE_AA)
    if ctr in doneFrames:
        cv2.putText(img,'frame:' + str(ctr) + " already done",(1,30), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('image',img)
    if moving: # if mouse is moving, then reload the image
        img = im2.copy()
    else:
        img = im3
    moving = False
    if buttonRelease:
        note = ""
        
        print([ix, iy, xx, yy])
        buttonRelease = False
        img = imgs[ctr].copy()
        im2 = img.copy()
        im3 = img.copy()
        ctr += 1
        while(1):
            k = cv2.waitKey(1) & 0xFF
            if k == ord('l'): # landing
                break
            elif k == ord("p"): # pollinating
                break
            elif k == ord("f"): # flying
                break
            elif k == ord("g"): # grooming
                break
            elif k == ord("a"): # add note
                note = input("Notes ")
                break
            elif k == ord('n'): # next frame
                break
            elif k == ord('b'): # back one frame
                ctr -= 1
                break
            elif k == ord("z"): # if there is no bee in frame
                note = "no bee in frame"
                ix, iy, xx, yy = np.nan, np.nan, np.nan, np.nan
                note = "no bee in frame"
                break
            elif k == 27:
                break  
                
                
        # refref: overwrite data if someone goes back        
        doneFrames = [rect[i][4] for i in range(len(rect))]
        whichFrames = [ctr == doneFrames[ii] for ii in range(len(doneFrames))]
        print(any(whichFrames))
        if any(whichFrames):
            doneIndex = [i for i, x in enumerate(whichFrames) if x]
            rect[doneIndex[0]] = [ix, iy, xx, yy, ctr, vidPath, chr(k), note]
        else:
            rect.append([ix, iy, xx, yy, ctr, vidPath, chr(k), note])
            
            

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break  
    elif k == ord("b"): # back one frame
        ctr -= 1
        img = imgs[ctr].copy()
        im2 = img.copy()
        im3 = img.copy()
    elif k == ord("z"): # if there is no bee in frame
        ctr += 1
        img = imgs[ctr].copy()
        im2 = img.copy()
        im3 = img.copy()
        note = "no bee in frame"
        rect.append([np.nan, np.nan, np.nan, np.nan, ctr, vidPath, chr(k), note])
cv2.destroyAllWindows()

pd.DataFrame(rect, columns = ["x1", "y1", "x2", "y2", "frameNum", "vidPath", "behavior", "notes"])

[470, 619, 702, 825]
False
[467, 602, 701, 811]
False
[458, 599, 694, 820]
False
[463, 608, 680, 839]
True
[472, 617, 711, 817]
False
[431, 555, 660, 823]
False
[505, 594, 664, 856]
False
[443, 599, 681, 842]
False
[449, 601, 687, 840]
False
[463, 594, 677, 822]
True
[457, 577, 676, 827]
True
[441, 481, 737, 706]
False
[449, 541, 692, 724]
False
[488, 487, 747, 754]
False


,x1,y1,x2,y2,frameNum,vidPath,behavior,notes
0,470.0,619.0,702.0,825.0,1,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,l,
1,467.0,602.0,701.0,811.0,2,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,l,
2,463.0,608.0,680.0,839.0,3,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,l,
3,472.0,617.0,711.0,817.0,4,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,l,
4,431.0,555.0,660.0,823.0,5,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,l,
5,505.0,594.0,664.0,856.0,6,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,l,
6,463.0,594.0,677.0,822.0,7,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,l,
7,457.0,577.0,676.0,827.0,8,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,l,
8,NaN,NaN,NaN,NaN,6,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,z,no bee in frame
9,NaN,NaN,NaN,NaN,7,D:\BeeTwoFlowerExperiment\2018_02_27__14_04_52...,z,no bee in frame


In [98]:
doneIndex[0]

7

In [81]:
doneFrames

[1]

In [ ]:
[rect[ctr-1][4] == doneFrames[ii] for ii in range(len(doneFrames))]

In [86]:

ctr

2

In [64]:
rect[ctr][4] in [rect[i][4] for i in range(len(rect))]

True

In [75]:
doneFrames = [rect[i][4] for i in range(len(rect))]

whichFrames = [rect[ctr][4] == doneFrames[ii] for ii in range(len(doneFrames))]
any(whichFrames)
doneIndex = [i for i, x in enumerate(whichFrames) if x]
doneIndex

[6, 9]

IndexError: list assignment index out of range

In [ ]:
img = np.zeros((512,512,3), np.uint8)
cv2.rectangle(img,(33 ,28),(496 ,494),(0,255,0),-1)
alpha = 0.1
overlay = img.copy()
cv2.addWeighted(overlay, alpha, img, 1-alpha, 0 )
cv2.imshow('image',overlay)

In [ ]:
img = imgs[0].copy()
im3 = img.copy()
alpha = 0.5
cv2.rectangle(im3, (33 ,28),(496 ,494),(0, 0, 255), -1)
cv2.addWeighted(im3, alpha, img, 1 - alpha,0, img)
cv2.imshow("output", img)

In [ ]:
cv2.rectangle(img,(ix,iy),(xx,yy),(0,255,0),2)
im2 = img.copy()
io.imshow(im2)

In [ ]:
# get user input
for ii in range(4):
    a = input("behavior")
    print(a)


In [36]:
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img,'frame:' + str(ctr),(1,30), font, 1,(255,255,255),2,cv2.LINE_AA)
cv2.imshow("img", img)

In [ ]:
# open dialog box

In [12]:
import tkinter as tk
from tkinter import simpledialog
def center_window(root, width=300, height=300):
    # get screen width and height
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    # calculate position x and y coordinates
    x = (screen_width/2) - (width/2)
    y = (screen_height/2) - (height/2)
    root.geometry('%dx%d+%d+%d' % (width, height, x, y))



application_window = tk.Tk()
center_window( application_window)


answer = simpledialog.askstring("Input", "What the behavior?",
                                parent=application_window)
if answer is not None:
    print("Your first name is ", answer)
else:
    print("You don't have a first name?")
application_window.destroy()



Your first name is  gg


In [ ]:
print("hi")